In [1]:
#!/usr/bin/env python
# -*- coding: gb18030 -*-
# @Time    : 2017/7/8 10:50
# @Author  : taoyongbo
# @Site    : 
# @File    : rankPrediction.py
# @desc    :
import csv
import multiprocessing
import os
import sys
import time

import numpy as np
import pandas as pd
from pandas import DataFrame
from scipy.spatial.distance import cdist
from sklearn import metrics
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.utils import shuffle
from sklearn.externals import joblib
import constant
import utils

In [3]:
tp = pd.read_csv('D:\\sogou\\brand\\canyinfuwu-feature', sep='\t', encoding='gb18030',
                         names=['brandName', 'category', 'cityCount', 'averageGrade', 'midgrade', 'averageCommentCount',
                                'midCommentCount', 'averageprice', 'midPrice', 'categoryTotalCount', 'fiveRank',
                                'tenRank', 'twentyRank', 'fortyRank', 'fivePercentRank', 'tenPercentRank',
                                'twentyPercentRank', 'fortyPercentRank', 'hundredPercentRank'],
                         quoting=csv.QUOTE_NONE)

In [4]:
def convert(value):
    value_log = np.log10(value + 1)
    return value_log

In [6]:
tp['cityCount_log'] = tp['cityCount'].apply(func=convert)
tp['averageGrade_log'] = tp['averageGrade']                                               # 保持不变
tp['averageCommentCount_log'] = tp['averageCommentCount'].apply(func=convert)
tp['averageprice_log'] = tp['averageprice'].apply(func=convert)
tp['categoryTotalCount_log'] = tp['categoryTotalCount'].apply(func=convert)

In [9]:
tp

,brandName,category,cityCount,averageGrade,midgrade,averageCommentCount,midCommentCount,averageprice,midPrice,categoryTotalCount,...,fivePercentRank,tenPercentRank,twentyPercentRank,fortyPercentRank,hundredPercentRank,cityCount_log,averageGrade_log,averageCommentCount_log,averageprice_log,categoryTotalCount_log
0,姜家烤鸡,餐饮服务,1,0,0,0,0,0,0,2,...,1.00,1.00,1.00,1.00,1.00,0.301030,0,0.000000,0.000000,0.477121
1,布拉提手感烘焙,餐饮服务,1,4,4,565,565,38,38,2,...,4.00,4.00,4.00,4.00,2.50,0.301030,4,2.752816,1.591065,0.477121
2,宽心园,餐饮服务,4,3,3,4,1,0,0,4,...,3.00,3.00,3.00,3.00,2.75,0.698970,3,0.698970,0.000000,0.698970
3,麦思贝儿,餐饮服务,76,3,3,26,6,14,11,120,...,3.50,3.25,2.62,2.31,1.54,1.886491,3,1.431364,1.176091,2.082785
4,丁大福鸡汤馄饨店,餐饮服务,1,3,3,120,120,15,15,2,...,3.00,3.00,3.00,3.00,2.50,0.301030,3,2.082785,1.204120,0.477121
5,奶茶先生,餐饮服务,28,4,3,144,5,10,10,34,...,4.00,3.50,3.29,2.71,1.82,1.462398,4,2.161368,1.041393,1.544068
6,帝王鸡,餐饮服务,1,0,0,0,0,0,0,1,...,1.00,1.00,1.00,1.00,1.00,0.301030,0,0.000000,0.000000,0.301030
7,食尚麻辣香锅,餐饮服务,17,3,3,143,5,23,22,21,...,4.00,4.00,3.80,3.44,2.24,1.255273,3,2.158362,1.380211,1.342423
8,MiCHA米迦千层．乳酪蛋糕,餐饮服务,1,3,3,2,2,0,0,2,...,3.00,3.00,3.00,3.00,3.00,0.301030,3,0.477121,0.000000,0.477121
9,重庆骑龙火锅,餐饮服务,3,4,3,399,269,46,42,8,...,4.00,4.00,4.00,3.25,2.12,0.602060,4,2.602060,1.672098,0.954243


In [11]:
tp.loc[:,['cityCount_log','averageGrade_log','averageCommentCount_log','averageprice_log','categoryTotalCount_log']]

,cityCount_log,averageGrade_log,averageCommentCount_log,averageprice_log,categoryTotalCount_log
0,0.301030,0,0.000000,0.000000,0.477121
1,0.301030,4,2.752816,1.591065,0.477121
2,0.698970,3,0.698970,0.000000,0.698970
3,1.886491,3,1.431364,1.176091,2.082785
4,0.301030,3,2.082785,1.204120,0.477121
5,1.462398,4,2.161368,1.041393,1.544068
6,0.301030,0,0.000000,0.000000,0.301030
7,1.255273,3,2.158362,1.380211,1.342423
8,0.301030,3,0.477121,0.000000,0.477121
9,0.602060,4,2.602060,1.672098,0.954243


In [12]:
dd = 1.041393

In [14]:
'%.2f' % dd

'1.04'

In [ ]:
import csv
import multiprocessing
import os
import time

import numpy as np
import pandas as pd
from sklearn.externals import joblib

import utils

logger = utils.get_logger('brandRankPrediction')
local_featurePoiRank_path = 'D:\structure\\featurePoi\globalrank\\'

local_model_path = 'D:\\sogou\\brand\\brandModel'

model_map = {
    "lr_binguanfandian.m": "宾馆饭店",
    "lr_jinrongyinxing.m": "金融银行",
    "lr_gouwuchangsuo.m": "购物场所",
    "lr_yiliaoweisheng.m": "医疗卫生",
    "lr_xuexiaokeyan.m": "学校科研",
    "lr_xiuxianyule.m": "休闲娱乐",
    "lr_tiyuchangguan.m": "体育场馆",
    "lr_qichefuwu.m": "汽车服务",
    "lr_gongsiqiye.m": "公司企业",
    "lr_canyinfuwu.m": "餐饮服务",
    "lr_qita.m": "其它",
    "lvyoujingdian.m": "旅游景点",
    "zhengfujiguan.m": "政府机关",
    "youzhengdianxin.m": "邮政电信",
    "changguanhuisuo.m": "场馆会所",
    "fangdichan.m": "房地产",
    "diming.m": "地名",
    "jiaotongchuxing.m": "交通出行",
    "xinwenmeiti.m": "新闻媒体",
}


def get_model_map(file_name):
    if file_name in model_map:
        return model_map.get(file_name)
    return None


def convert_level(y):
    """
    format rank level
    :param y: 
    :return: 
    """
    # if y < 1.5:
    #     y = 1
    # elif y < 2.5:
    #     y = 2
    # elif y < 3.5:
    #     y = 3
    # elif y < 4.5:
    #     y = 4
    # else:
    #     y = 5


    return '%.2f' % y


def load_models(model_file_list):
    model_dict = {}
    for model_file in model_file_list:
        file_name = os.path.basename(model_file)

        current_model = joblib.load(model_file)
        if file_name in model_map:
            key = model_map.get(file_name)
            model_dict.__setitem__(key, current_model)
    return model_dict


def files_rank_cluster(local_featurePoi_path, output_path=None):
    global local_featurePoiRank_path
    if output_path:
        local_featurePoiRank_path = output_path

    logger.info(local_featurePoiRank_path)
    begin_time = time.time()
    logger.info("all fileList handle process")

    model_file_list, model_dirList = utils.get_files(local_model_path)

    # load model
    model_dict = load_models(model_file_list)

    # 读取文件
    fileList, dirList = utils.get_files(local_featurePoi_path)

    logger.info('total file:{num}'.format(num=len(fileList)))
    pool = multiprocessing.Pool(processes=5)

    for file in fileList:
        # brand_rank_prediction(file, model_dict)
        pool.apply_async(brand_rank_prediction, (file, model_dict,))  #
    pool.close()
    pool.join()

    end_time = time.time()

    logger.info("all fileList handle finished,total time:{total_time} s".format(total_time=(end_time - begin_time)))


def convert(value):
    value_log = np.log10(value + 1)
    return value_log


def is_atm(str):
    if 'ATM' in str or '24小时自助银行' in str:
        return 0
    else:
        return 1


def feature_covert(tp):
    tp['cityCount_log'] = tp['cityCount'].apply(func=convert)
    tp['averageGrade_log'] = tp['averageGrade']  # 保持不变
    tp['averageCommentCount_log'] = tp['averageCommentCount'].apply(func=convert)
    tp['averageprice_log'] = tp['averageprice'].apply(func=convert)
    tp['categoryTotalCount_log'] = tp['categoryTotalCount'].apply(func=convert)
    tp['is_atm'] = tp['brandName'].apply(func=is_atm)
    return tp


def brand_rank_prediction(file, model_dict):
    file_name = os.path.basename(file)

    try:
        logger.info("{file_name} predict".format(file_name=file_name))
        predict_begin_time = time.time()

        tp = pd.read_csv(file, sep='\t', encoding='gb18030',
                         names=['brandName', 'category', 'cityCount', 'averageGrade', 'midgrade', 'averageCommentCount',
                                'midCommentCount', 'averageprice', 'midPrice', 'categoryTotalCount', 'fiveRank',
                                'tenRank', 'twentyRank', 'fortyRank', 'fivePercentRank', 'tenPercentRank',
                                'twentyPercentRank', 'fortyPercentRank', 'hundredPercentRank'],
                         quoting=csv.QUOTE_NONE)

        parent_category = tp['category'][0]

        if not parent_category in model_dict:
            brand_rank_result = tp.loc[:, ['brandName', 'hundredPercentRank', 'categoryTotalCount']]
        else:
            tp = feature_covert(tp)
            if file_name == 'binguanfandian-feature':
                features = tp.loc[:, ['averageCommentCount_log', 'averageprice_log']]
            elif file_name == 'jinrongyinxing-feature':
                features = tp.loc[:,
                           ['cityCount_log', 'averageGrade_log', 'averageCommentCount_log', 'averageprice_log',
                            'categoryTotalCount_log', 'is_atm']]
            else:
                features = tp.loc[:,
                           ['cityCount_log', 'averageGrade_log', 'averageCommentCount_log', 'averageprice_log',
                            'categoryTotalCount_log']]

            model = model_dict.get(parent_category)

            y_train_pred = model.predict(features)

            tp['y_train_pred'] = y_train_pred
            tp['pre_rank_level'] = tp['y_train_pred'].apply(func=convert_level)
            brand_rank_result = tp.loc[:, ['brandName', 'pre_rank_level', 'categoryTotalCount']]

        path = local_featurePoiRank_path + file_name + "-brandRank"
        brand_rank_result.to_csv(path, encoding='gb18030', sep='\t', index=False, header=False, quoting=csv.QUOTE_NONE)

        logger.info("{file_name} rank predict finished,used time:{use_time}".format(
            file_name=file_name, use_time=time.time() - predict_begin_time))


    except Exception as e:
        logger.info('{file_name} exception,info:{info}'.format(file_name=file_name, info=utils.print_exception()))


if __name__ == '__main__':
    files_rank_cluster('D:\sogou\\brand\\feature\\', 'D:\sogou\\brand\\output\\')

2017-12-14 18:01:10,601 - D:\sogou\brand\output\
2017-12-14 18:01:10,627 - all fileList handle process
2017-12-14 18:01:10,636 - total file:1


In [ ]:
features